In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from feature_engine.selection import DropCorrelatedFeatures
import joblib
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.neighbors import LocalOutlierFactor

class_weight parametresine şunlar atanabilir:

None: Bütün sınıflar eşit ağırlığa sahip olur. Bu varsayılan değerdir.
'balanced': Sınıfların frekanslarına göre otomatik olarak ağırlık atanır. Ağırlıklar, n_samples / (n_classes * np.bincount(y)) formülü ile hesaplanır. Burada n_samples toplam örnek sayısıdır ve np.bincount(y) ise her sınıftaki örnek sayısını verir.
{sınıf_etiketi: ağırlık}: Bu, sınıflar için elle ağırlık belirtmenize olanak tanır. Örneğin, iki sınıflı bir problem için {0: 1, 1: 50} şeklinde bir sözlük belirtilebilir.

In [2]:
def grab_cols(df):
    num_cols = list(df.select_dtypes(include="number"))
    cat_cols = [col for col in df.columns if col not in num_cols]
    num_but_cat = [col for col in num_cols if df[col].nunique()<10]
    cat_but_car = [col for col in cat_cols if df[col].nunique() >20]
    cat_cols = cat_cols + num_but_cat 
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    num_cols = [col for col in num_cols if col not in num_but_cat]
    print(f"cat_cols = {len(cat_cols)}")
    print(f"num_cols = {len(num_cols)}")
    print(f"num_but_cat = {len(num_but_cat)}")
    print(f"cat_but_car= {len(cat_but_car)}")
    return cat_cols,num_cols,cat_but_car,num_but_cat

In [3]:
def diabetes_data_prep():
    df = pd.read_csv("diabetes.csv")
    df.columns = [col.lower() for col in df.columns]
    X = df.drop("outcome",axis=1)
    y =df["outcome"]
    cols=["glucose","bloodpressure","skinthickness","insulin","bmi"]

    for col in cols:
        X.loc[X[col]==0,col] = np.nan
    def outliers(df,variable):
        q1= df[variable].quantile(0.2)
        q3 = df[variable].quantile(0.8)
        iqr = q3 - q1
        lower_lim = q1 - 1.5*iqr
        upper_lim = q3 + 1.5*iqr
        return lower_lim,upper_lim

    def replace_outliers(X,col):
        lower_lim,upper_lim = outliers(X,col)
        X[col].clip(lower=lower_lim,upper=upper_lim,inplace=True)

    replace_outliers(X,"insulin")
    imp_missforest = IterativeImputer(
    estimator=XGBRegressor(n_estimators=300,max_depth=5),
    max_iter=30,
    initial_strategy="median",
    random_state=0
    ).set_output(transform="pandas")

    X=imp_missforest.fit_transform(X)
    def ohe(dataframe,cat_cols):
        dataframe = pd.get_dummies(dataframe,columns=cat_cols,drop_first=True,dtype=int)
        return dataframe
    X["new_glucose_cat"] = pd.cut(x=X["glucose"],bins=[-1,100,140,200],labels=["normal","prediabetes","danger"])

    X.loc[X["age"]<32,"new_age_cat"] = 0
    X.loc[(X["age"]>=32) & (X["age"]<=50),"new_age_cat"]= 1
    X.loc[X["age"]>50,"new_age_cat"] =2

    # X["new_age2"] = pd.cut(x=X["age"],bins=[-1,32,50,100],labels= [0,1,2]) # alt sınıfa dahil eder

    X["new_bmi"] = pd.cut(x=X["bmi"],bins=[-1,18.5,24.9,29.9,100],labels=["underweight","healthy","overweight","obese"])
    X["new_bloodpressure"] = pd.cut(x=X["bloodpressure"],bins=[-1,79,89,123],labels=["normal","hs1","hs2"])
    
    cat_cols,num_cols,cat_but_car,num_but_cat = grab_cols(X)
    X=ohe(X,cat_cols)
    lof = LocalOutlierFactor(n_neighbors=10,n_jobs=-1)
    lof.fit_predict(X)
    X_scores = lof.negative_outlier_factor_
    df = pd.concat([X,y],axis=1)
    df=df.drop(labels =list(df[X_scores<-1.8].index),axis=0 )
    X=df.drop("outcome",axis=1)
    y = df["outcome"]
    sc = StandardScaler().set_output(transform="pandas")
    X = sc.fit_transform(X)
    return X,y
X,y = diabetes_data_prep()

c:\Users\cagat\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


cat_cols = 4
num_cols = 8
num_but_cat = 1
cat_but_car= 0


In [4]:
lr = LogisticRegression(class_weight='balanced')
cross_val_score(lr,X,y,cv=10,n_jobs=-1,scoring="recall").mean()

0.7804843304843305

In [5]:
cross_val_score(lr,X,y,cv=10,n_jobs=-1,scoring="roc_auc").mean()

0.841465724751439

In [9]:
cross_val_score(lr,X,y,cv=10,n_jobs=-1,scoring="accuracy").mean()

0.7618421052631579

In [6]:
rf = RandomForestClassifier(class_weight='balanced',n_estimators=200,max_depth=4)
cross_val_score(rf,X,y,cv=10,n_jobs=-1,scoring="recall").mean()

0.7652421652421653

In [8]:
cross_val_score(rf,X,y,cv=10,n_jobs=-1,scoring="roc_auc").mean()

0.8434630501773359

In [4]:
X

,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,new_glucose_cat_prediabetes,new_glucose_cat_danger,new_bmi_healthy,new_bmi_overweight,new_bmi_obese,new_bloodpressure_hs1,new_bloodpressure_hs2,new_age_cat_1.0,new_age_cat_2.0
0,0.633702,0.869437,-0.036565,0.667253,0.070035,0.186704,0.485557,1.458321,-0.953702,1.725995,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,1.445486,-0.338186
1,-0.851843,-1.208138,-0.544486,0.030784,-0.675924,-0.864077,-0.363975,-0.186077,-0.953702,-0.579376,-0.391488,1.795055,-1.290994,-0.497943,-0.290113,-0.691809,-0.338186
2,1.227920,2.023646,-0.713794,-0.954370,0.744538,-1.359445,0.624067,-0.099529,-0.953702,1.725995,2.554360,-0.557086,-1.290994,-0.497943,-0.290113,1.445486,-0.338186
3,-0.851843,-1.076228,-0.544486,-0.605686,-0.637236,-0.638909,-0.930330,-1.051549,-0.953702,-0.579376,-0.391488,1.795055,-1.290994,-0.497943,-0.290113,-0.691809,-0.338186
4,-1.148952,0.506686,-2.745481,0.667253,0.223966,1.612764,5.598139,-0.012982,1.048545,-0.579376,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,1.445486,-0.338186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,1.822138,-0.680500,0.302050,2.046271,0.363620,0.081626,-0.918018,2.583435,1.048545,-0.579376,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,-0.691809,2.956956
764,-0.554734,0.012025,-0.205872,-0.181373,0.500327,0.667061,-0.397833,-0.532265,1.048545,-0.579376,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,-0.691809,-0.338186
765,0.336593,-0.020952,-0.036565,-0.605686,-0.427754,-0.924121,-0.690245,-0.272624,1.048545,-0.579376,-0.391488,1.795055,-1.290994,-0.497943,-0.290113,-0.691809,-0.338186
766,-0.851843,0.143935,-1.052408,0.025221,-0.286800,-0.338686,-0.370131,1.198679,1.048545,-0.579376,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,1.445486,-0.338186


In [14]:
496/264

1.878787878787879

In [15]:
rf_params={"max_depth":[3,4,5,6], 
           "min_samples_split":[15,20],
           "n_estimators":[200,300]}

xgb_params = {"booster":["gblinear","gbtree"],
              "n_estimators":[200,300],
              "reg_lambda":[0.02,0.05],
              "reg_alpha":[0.01,0.02]}

lr_params = {'C': [0.01, 0.1, 1, 10],
            'penalty': ['l1', 'l2'],
            "max_iter":[5000,1000]}


classifiers = [("rf",RandomForestClassifier(class_weight='balanced'),rf_params),
               ("xgb",XGBClassifier(objective ="binary:logistic",scale_pos_weight=1.88),xgb_params),
               ("lr",LogisticRegression(solver='liblinear',class_weight='balanced'),lr_params)]

def hyperparameter_optimization(X,y,scoring="roc_auc"):
    print("hyperparameter optimization")
    best_models ={}
    for name,classifier,params in classifiers:
        print(f"##### {name}######")
        cv_results = cross_val_score(classifier,X,y,scoring=scoring,cv=10,n_jobs=-1).mean()
        print(f"{scoring} (Before): {cv_results}")
        
        gs = GridSearchCV(classifier,params,cv=10,scoring=scoring).fit(X,y)
        final_model = classifier.set_params(**gs.best_params_)
        
        cv_results = cross_val_score(final_model,X,y,scoring=scoring,cv=10,n_jobs=-1).mean()
        print(f"{scoring} (After): {cv_results}")
        print(f"{name} best_params: {gs.best_params_}", end="\n\n")
        best_models[name] = final_model
    return best_models
    
best_models = hyperparameter_optimization(X,y,scoring="roc_auc")

hyperparameter optimization
##### rf######
roc_auc (Before): 0.8283322286179429
roc_auc (After): 0.8426249200534913
rf best_params: {'max_depth': 5, 'min_samples_split': 15, 'n_estimators': 300}

##### xgb######
roc_auc (Before): 0.789438862724577
roc_auc (After): 0.8463807779522066
xgb best_params: {'booster': 'gblinear', 'n_estimators': 200, 'reg_alpha': 0.01, 'reg_lambda': 0.05}

##### lr######
roc_auc (Before): 0.841388801674516
roc_auc (After): 0.8460824466538753
lr best_params: {'C': 0.1, 'max_iter': 1000, 'penalty': 'l1'}



In [16]:
best_models

{'rf': RandomForestClassifier(class_weight='balanced', max_depth=5,
                        min_samples_split=15, n_estimators=300),
 'xgb': XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=200, n_jobs=None, num_parallel_tree=None,
               predictor=None, random_state=None, ...),
 'lr': LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000, penalty='l1',
   

In [17]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [("lr",best_models["lr"]),
                                            ("rf",best_models["rf"]),
                                            ("xg",best_models["xgb"])],
                              voting='soft',
                            weights=[1,1,1])

cross_val_score(voting_clf,X,y,cv=10,n_jobs=-1,scoring="roc_auc").mean()

0.8499018547589976

In [18]:
cross_val_score(voting_clf,X,y,cv=10,n_jobs=-1,scoring="accuracy").mean()

0.7776315789473685

In [19]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [("lr",best_models["lr"]),
                                            ("rf",best_models["rf"]),
                                            ("xg",best_models["xgb"])],
                              voting='hard')

cross_val_score(voting_clf,X,y,cv=10,n_jobs=-1,scoring="accuracy").mean()

0.7749999999999999

In [20]:
lr = LogisticRegression()
xg = XGBClassifier(n_estimators=100,booster="gblinear",reg_lambda=0.02,reg_alpha=0.01,objective ="binary:logistic")

voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('xg', xg)],
    voting='soft',
    weights=[4,1]  # Ağırlıkları belirtiyoruz
)
cross_val_score(voting_clf,X,y,cv=10,n_jobs=-1,scoring="accuracy").mean()

0.7776315789473685

In [21]:
X

,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,new_glucose_cat_prediabetes,new_glucose_cat_danger,new_bmi_healthy,new_bmi_overweight,new_bmi_obese,new_bloodpressure_hs1,new_bloodpressure_hs2,new_age_cat_1.0,new_age_cat_2.0
0,0.633702,0.869437,-0.036565,0.667253,0.070035,0.186704,0.485557,1.458321,-0.953702,1.725995,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,1.445486,-0.338186
1,-0.851843,-1.208138,-0.544486,0.030784,-0.675924,-0.864077,-0.363975,-0.186077,-0.953702,-0.579376,-0.391488,1.795055,-1.290994,-0.497943,-0.290113,-0.691809,-0.338186
2,1.227920,2.023646,-0.713794,-0.954370,0.744538,-1.359445,0.624067,-0.099529,-0.953702,1.725995,2.554360,-0.557086,-1.290994,-0.497943,-0.290113,1.445486,-0.338186
3,-0.851843,-1.076228,-0.544486,-0.605686,-0.637236,-0.638909,-0.930330,-1.051549,-0.953702,-0.579376,-0.391488,1.795055,-1.290994,-0.497943,-0.290113,-0.691809,-0.338186
4,-1.148952,0.506686,-2.745481,0.667253,0.223966,1.612764,5.598139,-0.012982,1.048545,-0.579376,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,1.445486,-0.338186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,1.822138,-0.680500,0.302050,2.046271,0.363620,0.081626,-0.918018,2.583435,1.048545,-0.579376,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,-0.691809,2.956956
764,-0.554734,0.012025,-0.205872,-0.181373,0.500327,0.667061,-0.397833,-0.532265,1.048545,-0.579376,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,-0.691809,-0.338186
765,0.336593,-0.020952,-0.036565,-0.605686,-0.427754,-0.924121,-0.690245,-0.272624,1.048545,-0.579376,-0.391488,1.795055,-1.290994,-0.497943,-0.290113,-0.691809,-0.338186
766,-0.851843,0.143935,-1.052408,0.025221,-0.286800,-0.338686,-0.370131,1.198679,1.048545,-0.579376,-0.391488,-0.557086,0.774597,-0.497943,-0.290113,1.445486,-0.338186


In [24]:
lr = LogisticRegression(class_weight='balanced')
cross_val_score(lr,X,y,cv=10,n_jobs=-1,scoring="accuracy").mean()

0.7618421052631579

In [25]:
def voting_classifier(best_models,X,y):
    voting_clf=VotingClassifier(estimators = [("lr",best_models["lr"]),
                                            ("rf",best_models["rf"]),
                                            ("xg",best_models["xgb"])],
                              voting='soft',
                            weights=[1,1,1])
    cv_results = cross_validate(voting_clf,X,y,cv=10,scoring=["accuracy","roc_auc","recall"])
    print(f"accuracy: {cv_results['test_accuracy'].mean()}")
    print(f"recall: {cv_results['test_recall'].mean()}")
    print(f"roc_auc: {cv_results['test_roc_auc'].mean()}")
    return voting_clf

In [26]:
cross_val_score(best_models["xgb"],X,y,cv=10,n_jobs=-1,scoring="accuracy").mean()

0.7776315789473685

In [27]:
y.value_counts()

outcome
0    496
1    264
Name: count, dtype: int64

In [28]:
# Sınıf sayıları
num_class_0 = 496
num_class_1 = 264

# Toplam örnek sayısı
total_samples = num_class_0 + num_class_1

# Ağırlıkları hesapla
weight_class_0 = total_samples / (2 * num_class_0)
weight_class_1 = total_samples / (2 * num_class_1)

print("0 sınıfı için ağırlık:", weight_class_0)
print("1 sınıfı için ağırlık:", weight_class_1)

0 sınıfı için ağırlık: 0.7661290322580645
1 sınıfı için ağırlık: 1.4393939393939394


In [29]:
496/264 # scale pos weight scale_pos_weight=1.88)

1.878787878787879

In [47]:
import xgboost as xgb
num_class_0 = 496
num_class_1 = 264

# Toplam örnek sayısı
total_samples = num_class_0 + num_class_1

# Ağırlıkları hesapla
weight_class_0 = total_samples / (2 * num_class_0)
weight_class_1 = total_samples / (2 * num_class_1)

# Y dizisine göre örnek ağırlıklarını oluştur
sample_weights = np.where(y == 0, weight_class_0, weight_class_1)

# XGBoost modelini eğit
dtrain = xgb.DMatrix(X, label=y, weight=sample_weights)
param = {'objective': 'binary:logistic'}
bst = xgb.train(param, dtrain)

In [ ]:
sample_weights

In [20]:
weight_class_0 = 1.43939394
weight_class_1 = 0.76612903

scale_pos_weight_value = weight_class_0 / weight_class_1
scale_pos_weight_value

1.8787878851164275

In [32]:
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score

def xgb_cross_val_score(X, y, params, num_rounds=50, n_folds=10):
    # Ağırlıkları hesapla
    num_class_0 = np.sum(y == 0)
    num_class_1 = np.sum(y == 1)
    total_samples = len(y)

    weight_class_0 = total_samples / (2 * num_class_0)
    weight_class_1 = total_samples / (2 * num_class_1)

    # Y dizisine göre örnek ağırlıklarını oluştur
    sample_weights = np.where(y == 0, weight_class_0, weight_class_1)

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    auc_scores = []
    
    for train_idx, val_idx in skf.split(X, y):
        dtrain_weight = sample_weights[train_idx]
        dval_weight = sample_weights[val_idx]

        dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx], weight=dtrain_weight)
        dval = xgb.DMatrix(X.iloc[val_idx], label=y.iloc[val_idx], weight=dval_weight)

        bst = xgb.train(params, dtrain, num_rounds)
        preds_val = bst.predict(dval)
        
        # AUC değerini hesapla
        auc = roc_auc_score(y.iloc[val_idx], preds_val)
        auc_scores.append(auc)
    
    return np.mean(auc_scores)

# Örnek parametreler
params = {'objective': 'binary:logistic'}

# Çapraz doğrulama AUC skorunu al
mean_auc = xgb_cross_val_score(X, y, params)
print(f"Mean AUC: {mean_auc}")



Mean AUC: 0.8067352753067037


In [52]:
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score, recall_score

def xgb_cross_val_score(X, y, params, num_rounds=100, n_folds=10):
    # Ağırlıkları hesapla
    num_class_0 = np.sum(y == 0)
    num_class_1 = np.sum(y == 1)
    total_samples = len(y)

    weight_class_0 = total_samples / (2 * num_class_0)
    weight_class_1 = total_samples / (2 * num_class_1)

    # Y dizisine göre örnek ağırlıklarını oluştur
    sample_weights = np.where(y == 0, weight_class_0, weight_class_1)

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    auc_scores = []
    recall_scores = []

    for train_idx, val_idx in skf.split(X, y):
        dtrain_weight = sample_weights[train_idx]
        dval_weight = sample_weights[val_idx]

        dtrain = xgb.DMatrix(X.iloc[train_idx], label=y.iloc[train_idx], weight=dtrain_weight)
        dval = xgb.DMatrix(X.iloc[val_idx], label=y.iloc[val_idx], weight=dval_weight)

        bst = xgb.train(params, dtrain, num_rounds)
        preds_val = bst.predict(dval)

        # AUC değerini hesapla
        auc = roc_auc_score(y.iloc[val_idx], preds_val)
        auc_scores.append(auc)

        # Recall değerini hesapla
        # Öncelikle tahmin edilen sınıfları elde edin (eşik değeri 0.5 olarak alındı)
        predicted_classes = np.where(preds_val >= 0.5, 1, 0)
        recall = recall_score(y.iloc[val_idx], predicted_classes)
        recall_scores.append(recall)

    return np.mean(auc_scores), np.mean(recall_scores)

# Belirtilen parametreler
params = {
    'booster': 'gblinear',
    'reg_lambda': 0.02,
    'reg_alpha': 0.01,
    'objective': 'binary:logistic'
}

# Çapraz doğrulama AUC ve Recall skorlarını al
mean_auc, mean_recall = xgb_cross_val_score(X, y, params)
print(f"Mean AUC: {mean_auc}")
print(f"Mean Recall: {mean_recall}")


Mean AUC: 0.8471467527181813
Mean Recall: 0.7649572649572649


multiclass

weights = [weight_class_0 if i == 0 else weight_class_1 if i == 1 else weight_class_2 for i in y]
dtrain = xgb.DMatrix(X, label=y, weight=weights)

Burada weight_class_0, weight_class_1 ve weight_class_2 her sınıf için belirlediğiniz ağırlıklardır


In [23]:
xg = XGBClassifier(n_estimators=100,booster="gblinear",reg_lambda=0.02,reg_alpha=0.01,objective ="binary:logistic"
                   ,scale_pos_weight=1.88)
cross_val_score(xg,X,y,cv=10,n_jobs=-1,scoring="recall").mean()

0.7692307692307692

In [24]:
cross_val_score(xg,X,y,cv=10,n_jobs=-1,scoring="roc_auc").mean()

0.8449379615093902

In [34]:

class_counts = np.bincount(y)


minority_class = np.argmin(class_counts)  # Azınlık sınıfını bulma
majority_class = 1 - minority_class  # Çoğunluk sınıfını bulma

# Ağırlıkları ayarlama
weights = {minority_class: 5, majority_class: 1}
sample_weights = [weights[label] for label in y]

# XGBoost modelini bu örnek ağırlıklarıyla eğit
xg2 = XGBClassifier(n_estimators=100,booster="gblinear",reg_lambda=0.02,reg_alpha=0.01,objective ="binary:logistic"
                   ,sample_weight=sample_weights)
cross_val_score(xg2,X,y,cv=10,n_jobs=-1,scoring="recall").mean()

0.556980056980057

In [ ]:
num_class_0 = 496
num_class_1 = 264

# Toplam örnek sayısı
total_samples = num_class_0 + num_class_1

# Ağırlıkları hesapla
weight_class_0 = total_samples / (2 * num_class_0)
weight_class_1 = total_samples / (2 * num_class_1)

# Y dizisine göre örnek ağırlıklarını oluştur
sample_weights = np.where(y == 0, weight_class_0, weight_class_1)
sample_weights

In [54]:
xg2 = XGBClassifier(n_estimators=100, booster="gblinear", reg_lambda=0.02, reg_alpha=0.01, objective="binary:logistic")
fit_params = {'sample_weight': sample_weights}
cross_val_score(xg2, X, y, cv=10, n_jobs=-1, scoring="recall", fit_params=fit_params).mean()

0.7769230769230769

In [45]:
xg2 = XGBClassifier(n_estimators=100,booster="gblinear",reg_lambda=0.02,reg_alpha=0.01,objective ="binary:logistic"
                   ,sample_weight=[0,5])
cross_val_score(xg2,X,y,cv=10,n_jobs=-1,scoring="recall").mean()

0.556980056980057

In [27]:
np.unique(y)

array([0, 1], dtype=int64)

In [28]:
voting_classifier(best_models,X,y)

accuracy: 0.7644736842105264
recall: 0.5568376068376069
roc_auc: 0.8476168963311821


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=0.1, max_iter=1000,
                                                 penalty='l1',
                                                 solver='liblinear')),
                             ('rf',
                              RandomForestClassifier(max_depth=6,
                                                     min_samples_split=15,
                                                     n_estimators=200)),
                             ('xg',
                              XGBClassifier(base_score=None, booster='gblinear',
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=No...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...))],
                 voting='soft', weights=[1, 1, 1])

In [29]:
def main():
    X,y = diabetes_data_prep()
    best_models = hyperparameter_optimization(X,y,scoring="roc_auc")